In [1]:
import io
import subprocess
import re
import pickle
import random

# TPTP Formatting
fof(name,formula_type,statement)
- name is the name of the formula
- formula_type is either axiom or conjecture, for our purposes.
- Axioms are facts
- Conjectures are what we want to prove/disprove


Extra TPTP notes:
- ! is the universal quantifier
- ? is the existential quantifier
- variables are denoted with capital letters
- func(C) is a function with variable C
- => is the implication operator
- & is AND
- | is OR
- ~ is NEGATION
- <=> is EQUIVALENCE
- = is EQUALITY
- != is INEQUALITY

In [2]:
class Axiom:
    def __init__(self,name,statement):
        self.name = name
        self.statement = statement
    def __str__(self):
        return "fof("+str(self.name)+",axiom,"+str(self.statement)+")."

class Conjecture:
    def __init__(self,name,statement):
        self.name = name
        self.statement = statement
    def __str__(self):
        return "fof("+str(self.name)+",conjecture,"+str(self.statement)+")."

In [ ]:
theorum = str(Axiom("1","b"))+str(Axiom("2","(a&b)=>b"))+str(Axiom("3","b=>c"))+str(Conjecture("c1","c"))

print(str(Axiom("1","a")))


In [ ]:
#needs E, can't run on colab
file_path = "f.tptp"
eprover_path = "/home/anmarch/source/eprover/PROVER/eprover"
with io.open(file_path,'w',encoding='utf-8') as f:
    f.write(theorum)

result = subprocess.run([eprover_path, "--proof-object", str(file_path)], capture_output=True)
output = result.stdout.decode()

if result.returncode == 0:
    #proof found
    print(output)
    pass

elif result.returncode == 1:
    #proof not found
    print(output)
    pass

else:
    #something else happened
    print(result)
    raise Exception("Something unexpected occured")

# Parse the proof output.
Todo: get rid of unnecessary info like filenames

In [ ]:
parsed_result = ""
for line in output.split('\n'):
    if len(line) > 0 and line[0] == '#':
        pass
    else:
        line = line.replace(' ','')
        print(line)
        break
        

# Generate expressions using:
- and &
- or |
- not ~
- implies =>
- a1,a2,a3 etc for axiom names
- c1,c2,c3 etc for conjecture names


In [ ]:
operation_list = ['&','|']

original = ['a','b','~a','~b',]
prop_names = []

for x in original:
    prop_names.append(str(x))

ops = operation_list
new = dict()
new2 = list()


for name in prop_names:
    new[str(name)]=str(name)

#i here is the length of the LHS of the axiom names, for example i=0 a=>a, i=1, a&a=>a, i=2 a&a|a=>a 
for i in range(1):
    for var1 in original:
        for var2 in prop_names:
            for o in ops:
                #may need regular expressions here to remove duplicated elements
                new[str(var1)+str(o)+str(var2)]=str(var1)+str(o)+str(var2)
        
    for e in new:
        original.append(str(new[e]))

for element in new:
    for name in prop_names:
        result = '('+str(new[element])+")=>"+str(name)
        new2.append(result)

In [ ]:
#no need to run this on colab
print(len(new2))
with open('data.pickle','wb') as f:
    pickle.dump(new2,f)

with open('data.pickle','rb') as f:
    data = pickle.load(f)

for i in range(10):
    print(data[random.randint(0,(len(data))-1)])

In [ ]:
statements = new2
conclusions = prop_names

statement_list = list()
conclusion_list = list()


for i in range(len(statements)):
    statement_list.append(Axiom(str(i),statements[i]))
for i in range(len(conclusions)):
    conclusion_list.append(Conjecture(str(i),conclusions[i]))

theorum_list = list()
for s in statement_list:
    for c in conclusion_list:
        theorum_list.append(str(s)+str(c))


#needs E, can't run on colab
file_path = "f.tptp"
eprover_path = "/home/anmarch/source/eprover/PROVER/eprover"
found_proofs = list()
unfound_proofs = list()
for t in theorum_list:
    with io.open(file_path,'w',encoding='utf-8') as f:
        f.write(str(t))

    result = subprocess.run([eprover_path, "--proof-object", str(file_path)], capture_output=True)
    output = result.stdout.decode()

    if result.returncode == 0:
        #proof found
        #print(output)
        found_proofs.append([str(t),result])
        pass

    elif result.returncode == 1:
        #proof not found
        unfound_proofs.append([str(t),result])
        #print(output)
        pass

    else:
        #something else happened
        print(result)
        raise Exception("Something unexpected occured")

In [ ]:
print(len(found_proofs))
print(len(unfound_proofs))
print(len(found_proofs)+len(unfound_proofs))
print(len(theorum_list))

# CNF creation
- & is AND
- | is OR
- ~ is NOT
- CNF -> (Disjunction) & CNF
- CNF -> (Disjunction)
- Disjunction -> Literal | Disjunction
- Disjunction -> Literal
- Literal -> ~Variable
- Literal -> Variable


In [14]:
def produce_disjunctions(literal_list):
    disjunction_dict = dict()
    disjunction_dict_old = dict()
    disjunction_list = []

    for l in literal_list:
        disjunction_dict[str(l)]=l
        disjunction_dict_old[str(l)]=l

    for key in disjunction_dict_old:
        for l in literal_list:
            disjunction_dict[str(l)+'|'+str(disjunction_dict_old[key])]=str(l)+'|'+str(disjunction_dict_old[key])

    for key in disjunction_dict:
        disjunction_list.append(disjunction_dict[key])

    disjunction_dict = None
    disjunction_dict_old = None
    
    return disjunction_list

def produce_cnf(disjunction_list):
    cnf_dict_new = dict()
    cnf_list_old = []
    cnf_list_new = []

    for d in disjunction_list:
        cnf_list_old.append('('+str(d)+')')
        cnf_dict_new['('+str(d)+')']='('+str(d)+')'

    for c in cnf_list_old:
        for d in disjunction_list:
            cnf_dict_new['('+str(d)+')&'+str(c)]='('+str(d)+')&'+str(c)
    
    for key in cnf_dict_new:
        cnf_list_new.append(cnf_dict_new[key])

    cnf_list_old = None
    cnf_dict_new = None

    return cnf_list_new

#variable_list = ['a','b','c', 'd', 'e']
variable_list = ['a','b','c']
literal_list = []
disjunction_list = []


for var in variable_list:
    for i in range(2):
        if i % 2 == 0:
            literal_list.append(str(var))
        else:
            literal_list.append('~'+str(var))

d_list = produce_disjunctions(literal_list)
cnf_list = produce_cnf(d_list)
print(len(d_list))
print(len(cnf_list))

42
1806


In [ ]:
statements = list()
conclusions = list()
for e in cnf_list:
    statements.append(e)
    conclusions.append(e)

statement_list = list()
conclusion_list = list()

#add a copy for yourself not in E format
for i in range(len(statements)):
    statement_list.append(Axiom(str(i),statements[i]))
for i in range(len(conclusions)):
    conclusion_list.append(Conjecture(str(i),conclusions[i]))

In [ ]:
theorum_list = list()
for s in statement_list:
    for c in conclusion_list:
        theorum_list.append(str(s)+str(c))

#for s1 in statement_list:
#    for s2 in statement_list:
#        if s1 is not s2:
#            for c in conclusion_list:
#               theorum_list.append(str(s1)+str(s2)+str(c))

#for s1 in statement_list:
#    for s2 in statement_list:
#        if s1 is not s2:
#            for s3 in statement_list:
#                if (s3 is not s1) and (s3 is not s2):
#                    for c in conclusion_list:
#                       theorum_list.append(str(s1)+str(s2)+str(s3)+str(c))

#needs E, can't run on colab
file_path = "f.tptp"
eprover_path = "/home/anmarch/source/eprover/PROVER/eprover"
found_proofs = list()
unfound_proofs = list()
for t in theorum_list:
    with io.open(file_path,'w',encoding='utf-8') as f:
        f.write(str(t))

    result = subprocess.run([eprover_path, "--proof-object", str(file_path)], capture_output=True)
    output = result.stdout.decode()

    if result.returncode == 0:
        #proof found
        #print(output)
        found_proofs.append([str(t),result])
        pass

    elif result.returncode == 1:
        #proof not found
        unfound_proofs.append([str(t),result])
        #print(output)
        pass

    else:
        #something else happened
        print(result)
        raise Exception("Something unexpected occured")

In [9]:
statement_list = list()
conclusion_list = list()


for i in range(len(cnf_list)):
    statement_list.append(Axiom(str(i),cnf_list[i]))
for i in range(len(cnf_list)):
    conclusion_list.append(Conjecture(str(i),cnf_list[i]))

print(len(statement_list))
print(len(conclusion_list))

420
420


In [10]:
print(len(found_proofs))
print(len(unfound_proofs))

52368
124032


In [11]:
#no need to run this on colab
with open('found.pickle','wb') as f:
    pickle.dump(found_proofs,f)

with open('found.pickle','rb') as f:
    found = pickle.load(f)


#no need to run this on colab
with open('unfound.pickle','wb') as f:
    pickle.dump(unfound_proofs,f)

with open('unfound.pickle','rb') as f:
    unfound = pickle.load(f)

In [13]:
for i in range(5):
    print(found[random.randint(0,(len(found))-1)])
print()
for i in range(5):
    print(unfound[random.randint(0,(len(unfound))-1)])

['fof(368,axiom,(a|~a)&(~a|~b)).fof(131,conjecture,(~b|~a)&(~a|a)).', CompletedProcess(args=['/home/anmarch/source/eprover/PROVER/eprover', '--proof-object', 'f.tptp'], returncode=0, stdout=b"# Initializing proof state\n# Scanning for AC axioms\n#\n#cnf(i_0_4, negated_conjecture, (a)).\n#\n#cnf(i_0_5, negated_conjecture, (b)).\n##\n# Proof found!\n# SZS status Theorem\n# SZS output start CNFRefutation\nfof(131, conjecture, ((~(b)|~(a))&(~(a)|a)), file('f.tptp', 131)).\nfof(368, axiom, ((a|~(a))&(~(a)|~(b))), file('f.tptp', 368)).\nfof(c_0_2, negated_conjecture, ~(((~b|~a)&(~a|a))), inference(fof_simplification,[status(thm)],[inference(assume_negation,[status(cth)],[131])])).\nfof(c_0_3, negated_conjecture, (((a|b)&(~a|b))&((a|a)&(~a|a))), inference(distribute,[status(thm)],[inference(fof_nnf,[status(thm)],[inference(fof_nnf,[status(thm)],[c_0_2])])])).\nfof(c_0_4, plain, ((a|~a)&(~a|~b)), inference(fof_simplification,[status(thm)],[368])).\ncnf(c_0_5, negated_conjecture, (a|a), inferen

fof(368,axiom,(a|~a)&(~a|~b)).fof(131,conjecture,(~b|~a)&(~a|a)).

should be 

(a|~a)&(~a|~b) => (~b|~a)&(~a|a)

token list:
(
)
a
|
~
b
=>

write the tokenization, input that into a LSTM to classify theorums